# Cholera

In [1]:

import os
#os.environ['XLA_PYTHON_CLIENT_PREALLOCATE']='false'

import jax
import itertools
import numpy as onp

import jax.numpy as np
import ipywidgets as widgets
import pandas as pd

from jax.numpy.linalg import inv, pinv
from jax.scipy.optimize import minimize
from scipy.linalg import solve_discrete_are as dare
from jax import jit, grad
from IPython import display
from toolz.dicttoolz import valmap, itemmap
from itertools import chain
from functools import partial

from tqdm.notebook import tqdm
from tensorflow_probability.substrates import jax as tfp
tfpk = tfp.math.psd_kernels
import tensorflow_probability.substrates.jax.distributions as tfd
import tensorflow_probability.substrates.jax.bijectors as tfb

from pomps import *
from resampling import *
from filtering import *
from optim import *



import matplotlib.pyplot as plt
plt.style.use('matplotlibrc')
onp.set_printoptions(suppress=True)

2024-05-25 15:14:27.176637: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
dataset = pd.read_csv('data/dacca/dacca.csv', index_col=0).reset_index(drop=True)
ys = np.array(dataset['cholera.deaths'].values)
dataset = pd.read_csv('data/dacca/covars.csv', index_col=0).reset_index(drop=True)
dataset.index = pd.read_csv('data/dacca/covart.csv', index_col=0).reset_index(drop=True).squeeze()
dataset = dataset.reindex(onp.array([1891 + i*(1/240) for i in range(12037)])).interpolate()
covars = np.array(dataset.values)#[20:]
#gamma, m, rho, epsilon, omega, c, beta_trend, sigma, tau, bs, k, delta = 8.0, 9.238, 7.1, 0.7, onp.exp(-4.5), 0.0051, (-6.8*10**(-3)), 639.6, 0.23, np.array([6.5,11.4,2.1,9.0,8.6,9.5]), 3, 0.02
#gamma, m, rho, epsilon, omega, c, beta_trend, sigma, tau, bs, k, delta = 17.3, 0.057, 0, 9.8, onp.exp(-4.8), 0.999999, (-5.0*10**(-3)), 3.2, 0.25, np.array([1.2,6.2,-3.4,3.9,3.2,4.3]), 3, 0.02


gamma = 20.8
epsilon = 19.1
rho = 0
delta = 0.02
m = 0.06
c = np.array(1)
beta_trend = -0.00498
bs = np.array([0.747, 6.38, -3.44, 4.23, 3.33, 4.55])
sigma = 3.13 #3.13 # 0.77
tau = 0.23 
omega = onp.exp(-4.5)
omegas = np.log(np.array([0.184, 0.0786, 0.0584, 0.00917, 0.000208, 0.0124]))


theta = transform_thetas(gamma, m, rho, epsilon, omega, c, beta_trend, sigma, tau, bs, omegas)

2024-05-25 15:14:28.625401: W external/xla/xla/service/gpu/nvptx_compiler.cc:742] The NVIDIA driver's CUDA version is 12.1 which is older than the ptxas CUDA version (12.4.99). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [4]:
pfilter(theta, ys, 10000, covars, thresh=-1)

Array(3748.2502, dtype=float32)

In [5]:
mop(theta, ys, 10000, covars, alpha=0.5)

Array(3748.7583, dtype=float32)

In [6]:
n_trials = 100
J = 10000

def get_rand_theta():
    return transform_thetas(onp.random.uniform(10.00, 40.00),
                onp.random.uniform(0.03, 0.60), 
                         rho, 
                 onp.random.uniform(0.20, 30.00), 
                         omega, 
                         c, 
                 onp.random.uniform(-1.00, 0.00)*0.01, 
                 onp.random.uniform(1.00, 5.00), 
                 onp.random.uniform(0.10, 0.50), 
                 onp.random.uniform(0,8,size=6)+np.array([-4,0,-4,0,0,0]), 
                 onp.random.uniform(-10,0,size=6))

def get_sds():    
    lows = transform_thetas(10.00,0.03, rho, 0.20, omega, c, 
             -1.00*0.01, 1.00, 0.10, 
             onp.zeros(6)+onp.array([-4,0,-4,0,0,0]), 
            -10*onp.ones(6))
    highs = transform_thetas(40.00,0.60, rho, 30.00, omega, c, 
                 0.00, 5.00, 0.50, 
                 8*onp.ones(6)+onp.array([-4,0,-4,0,0,0]), 
                onp.zeros(6))
    return (highs-lows)/100



def get_rand_theta(J=J):
    lows = transform_thetas(10.00,0.03, rho, 0.20, omega, c, 
             -1.00*0.01, 1.00, 0.10, 
             onp.zeros(6)+onp.array([-4,0,-4,0,0,0]), 
            -10*onp.ones(6))
    highs = transform_thetas(40.00,0.60, rho, 30.00, omega, c, 
                 0.00, 5.00, 0.50, 
                 8*onp.ones(6)+onp.array([-4,0,-4,0,0,0]), 
                onp.zeros(6))
    rands = onp.array(onp.repeat(((lows+highs)/2)[None,:], J, axis=0)).T
    rands[~onp.isinf(lows)] = onp.random.uniform(lows[~onp.isinf(lows)], 
                       highs[~onp.isinf(highs)],
                       size=(J, len(highs[~onp.isinf(highs)]))).T
    return rands.T
                       
                       


sigmas = (np.abs(theta)/600)
theta_ests = theta + 60*sigmas*onp.random.normal(size=theta.shape) #6 for close to maxima
#theta_ests = get_rand_theta(J).mean(0)

'''
gd_logliks, gd_ests = train(theta_ests, ys, covars, beta=0.9, eta=5e-4/600,
                            verbose=True, itns=20, J=J, thresh=0, method='SGD', scale=False)


newt_logliks, newt_ests = train(theta_ests, ys, covars, beta=0.9, eta=1e-2, c=1e-1, max_ls_itn=5, 
                            verbose=True, itns=20, J=J, Jh=500, thresh=0, method='Newton', scale=True, ls=True)

'''
print(pfilter(theta_ests, ys, J, covars, thresh=-1))
#res = minimize(pfilter, x0=theta_ests, args=(ys, J, covars, 0), method='BFGS', options={'maxiter': 10})
get_thetas(theta_ests)

# try cross-comparison, fig 2 in pnas, on close to MLE
# use if2 settings

4309.2744


(Array(25.470907, dtype=float32),
 Array(0.06987456, dtype=float32),
 Array(0., dtype=float32),
 Array(13.268806, dtype=float32),
 Array(0.01020048, dtype=float32),
 Array(1., dtype=float32),
 Array(-0.00456467, dtype=float32),
 Array(2.9220684, dtype=float32),
 Array(0.24351847, dtype=float32),
 Array([ 0.91186714,  4.7945175 , -3.919889  ,  4.137083  ,  3.1338313 ,
         4.8495283 ], dtype=float32),
 Array([-1.8031788, -2.858297 , -3.1682634, -4.236103 , -7.828042 ,
        -4.006463 ], dtype=float32),
 3,
 0.02)

In [10]:
state = rinit(theta, 1, covars).squeeze()
t=0
covar = covars[t][None,:]
key = jax.random.PRNGKey(0)


S, I, Y, deaths, pts, t, count = state[0], state[1], state[2], state[3], state[4:-2], state[-2], state[-1]
t = t.astype(int)
trends, dpopdts, pops, seass = covar[:,0], covar[:,1], covar[:,2], covar[:,3:]
gamma, deltaI, rho, eps, omega, clin, beta_trend, sd_beta, tau, bs, omegas, nrstage, delta = get_thetas(theta)
dt = 1/240
deaths = 0
nrstage = 3
clin = 1 # HARDCODED SEIR
rho = 0 # HARDCODED INAPPARENT INFECTIONS
std = onp.sqrt(dt) #onp.sqrt(onp.sqrt(dt))

neps = eps*nrstage
rdeaths = np.zeros(nrstage)
passages = np.zeros(nrstage+1)

trend = trends[t]; dpopdt = dpopdts[t]; pop = pops[t]; seas = seass[t]
beta = np.exp(beta_trend*trend + np.dot(bs, seas))
omega = np.exp(np.dot(omegas, seas))

effI = I/pop
births = dpopdt + delta*pop # births
passages = passages.at[0].set(gamma*I) #recovery
ideaths = delta*I #natural i deaths
disease = deltaI*I #disease death
ydeaths = delta*Y #natural rs deaths
wanings = rho*Y #loss of immunity

for j in range(nrstage):
    rdeaths = rdeaths.at[j].set(pts[j]*delta) #natural R deaths
    passages = passages.at[j+1].set(pts[j]*neps) # passage to the next immunity class
    
sdeaths = delta*S # natural S deaths
for j in range(nrstage):
    pts = pts.at[j].add((passages[j] - passages[j+1] - rdeaths[j])*dt)

deaths += disease*dt # cumulative deaths due to disease


In [85]:

count += np.any(np.hstack([np.array([S, I, Y, deaths]), pts]) < 0)

S = np.clip(S, a_min=0); I = np.clip(I, a_min=0); Y = np.clip(Y, a_min=0)
pts = np.clip(pts, a_min=0); deaths = np.clip(deaths, a_min=0)

t += 1

In [629]:
dw = tfd.Normal(loc=0, scale=std, name='dw')
infections = tfd.TransformedDistribution(
    tfd.TransformedDistribution(
        tfd.TransformedDistribution(
            tfd.TransformedDistribution(
                tfd.TransformedDistribution(dw, tfb.Scale(scale=sd_beta/dt)),
                tfb.Shift(beta)),
            tfb.Scale(effI)),
        tfb.Shift(omega)),
    tfb.Scale(S))

dS = tfd.TransformedDistribution(
        tfd.TransformedDistribution(
            tfd.TransformedDistribution(infections, tfb.Scale(-1)),
        tfb.Shift(births-sdeaths+passages[nrstage]+wanings)),
    tfb.Scale(dt))
    
dI = tfd.TransformedDistribution(
        tfd.TransformedDistribution(
            tfd.TransformedDistribution(infections, tfb.Scale(clin)),
        tfb.Shift(-disease-ideaths-passages[0])),
    tfb.Scale(dt))


Sp = tfd.TransformedDistribution(dS, tfb.Shift(S))
Ip = tfd.TransformedDistribution(dI, tfb.Shift(I))

sim_nextstate = rproc(rinit(theta,1,covars)[0], theta, key, covars)
## Log-density of S', I'
Sp.log_prob(sim_nextstate[0]) + Ip.log_prob(sim_nextstate[1])

/home/kevin/anaconda3/envs/jax/lib/python3.10/site-packages/tensorflow_probability/python/internal/backend/jax/ops.py:339: UserWarning: Explicitly requested dtype <class 'numpy.float64'> requested in array is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  return np.array(value, dtype=dtype)


Array(-34.91888, dtype=float32)